Ein Skript zur Extrahierung der Versuchsdaten aus der Datenbank der Edge-Box

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta
import plotly.io as pio
from time import sleep

DB Config

In [ ]:
#####DB Config
host= "Dnedgev333"
port=27017

Set parameters

In [ ]:
["T_KWAV1",
"T_KWAR1_MS",
"T_KWR_MS",
"V_KW_MS",
"V_KWAV1",
"V_KWAV2",
"T_KSM1",
"T_SCHLITTEN_Y",
"T_SPI_MB1",
"T_SPI_MB2",
"T_RAUM",
"T_BETT_X",
"T_STAE_Y",
"T_BETT_Z",
"T_Spindel",
"ActSpeed_S",
"Power_X",
"Torque_X",
"Current_X",
"T_Motor_X",
"Power_Y",
"Torque_Y",
"Current_Y",
"T_Motor_Y",
"Power_Z",
"Torque_Z",
"Current_Z",
"T_Motor_Z",
"Power_S",
"Torque_S",
"Current_S",
"T_Motor_S",
"Power_A",
"Torque_A",
"Current_A",
"T_Motor_A",
"Power_B",
"Torque_B",
"Current_B",
"T_Motor_B"]

In [ ]:
save_path= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\Edgebox\\Versuchsdaten\\5s_data\\'#neue_Spindel\\'
test_nr= "TEST_NR"
csv_selected_columns= [
    test_nr,
    "T_KWAV1",
    "T_KWAR1_MS",
    "T_KWR_MS",
    "V_KW_MS",
    "V_KWAV1",
    "V_KWAV2",
    "T_KSM1",
    "T_SCHLITTEN_Y",
    "T_SPI_MB1",
    "T_SPI_MB2",
    "T_RAUM",
    "T_BETT_X",
    "T_STAE_Y",
    "T_BETT_Z",
    "T_Spindel",
    "ActSpeed_S",
    "Power_X",
    "Torque_X",
    "Current_X",
    "T_Motor_X",
    "Power_Y",
    "Torque_Y",
    "Current_Y",
    "T_Motor_Y",
    "Power_Z",
    "Torque_Z",
    "Current_Z",
    "T_Motor_Z",
    "Power_S",
    "Torque_S",
    "Current_S",
    "T_Motor_S",
    "Power_A",
    "Torque_A",
    "Current_A",
    "T_Motor_A",
    "Power_B",
    "Torque_B",
    "Current_B",
    "T_Motor_B",
    'kuelleistung']
columns_with_units= [
                        "T_BETT_X (C°)",
                        "T_Spindel (C°)",
                        "T_Motor_S (C°)",
                        "T_BETT_Z (C°)",
                        "Torque_S",
                        "ActSpeed_S",
                        "T_STAE_Y (C°)" ,
                        "Power_S" ,
                        "T_KWR_MS (C°)" ,
                        "T_SCHLITTEN_Y (C°)",
                        "T_RAUM(C°)",
                        "V_KW_MS ",
                        "Current_S",
                        "T_KWAR1_MS (C°)" ,
                        "T_KWAV1 (C°)" ,
                        "V_KWAV1",
                        "T_KSM1 (C°)" ,
                        "V_KWAV2" ,
                        "T_SPI_MB1 (C°)",
                        "T_SPI_MB2 (C°)",
                        'kuelleistung']

Set time priod needed

In [ ]:
start= datetime.strptime("2023-05-23T07:36:31.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime ("2023-05-24T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")

Get Data from database

In [ ]:
## Read raw and preprocessing data
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
prev_time= None
current_time= None
stop= False
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
        ### Get right data between start / end 
        if record['date']< start or record['date'] > end:
            continue
        current_time= record['date']+ timedelta(hours= +2)
        if prev_time is not None  and (current_time -prev_time).total_seconds()> 5.5:
            print('Period= ', (current_time -prev_time).total_seconds())
            print('prev_time', prev_time)
            print('current_time', current_time)
            print('#########################################################')
        for item in record['raw_data'].keys():
            record['raw_data'][item]= [np.round(record['raw_data'][item],decimals=2)]
        record['raw_data']['date']=[current_time]
        ####################################
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
        prev_time= current_time
signals.reset_index(inplace= True)
org_signal= signals.copy()

Filtering based on the test_Nr

In [ ]:
signals= org_signal#org_signal[org_signal[test_nr] ==2]#

In [ ]:
signals.columns

Calculate the power

In [ ]:
signals['kuelleistung']=  (signals['T_KWR_MS']-signals['T_KWAV1']) *  signals['V_KW_MS'] * 1.16 *60/1000

Visualization of the predictions and labels and Error borders (Needed from Andreas)

In [ ]:
# scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# # selected_Columns= ['t_bett','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7', prediction_variable, output_variable, 'db_prediction_abs_error']
# # y_axis_names= selected_Columns
# #print(preds)
# fig= make_subplots(rows=1,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
# ##Verlagerung
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable], name='Gemessene Verlagerung (Welle)', mode= scatter_mode), row= 1, col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[prediction_variable], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 1, col= 1)

# ##Restfehler
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable] - 1000 * signals[prediction_variable], name=' Restfehler', mode= scatter_mode),  row= 1 , col= 1)
# ## Draw the tolerence +-5
# fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 1 , col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 1 , col= 1)
# fig.update_yaxes(title_text= 'Verlagerung Micro-Meter', row= 1, col= 1)
# fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data, Versuch am 2023-01-11T20:00:00 ISO 230-3 Trocken')

Static Visualization

In [ ]:
singnals_avg= signals.mean()
singnals_avg

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= csv_selected_columns 
y_axis_names= selected_Columns
fig= make_subplots(rows=len(selected_Columns),cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.008, subplot_titles= [selected_Columns[i] +' Mean='+ str(np.round(singnals_avg[selected_Columns[i]],2)) for i in range(len(selected_Columns))])
for i in range(len(selected_Columns)):
    trace= selected_Columns[i]
    fig.add_trace(go.Scatter(x= signals['date'], y= signals[trace], name=trace , mode= scatter_mode), row= i+1, col= 1)
    #fig.update_yaxes(title_text= trace, row= i+1, col= 1)
fig.update_layout(height=3000, width=1200, title_text= 'Machine Data')
fig.show()

Save data to .csv file

In [ ]:
# ###Save to .CSV-File######
# csv_df= signals[csv_selected_columns]
# csv_df[output_variable]= csv_df[output_variable].round(6)
# csv_df[csv_selected_columns].to_csv(str(save_path+'Versuch20_01_2023_trocken_Fraesen.csv'))